In [ ]:
#Create table
# subj, session, chanel name (ncs file), recording site, name of area, single/multi unit, sampling freq, stat...
#the table is used in the future analysis

In [2]:
import database.neuralynx_extract.combinato_tools as ct
import database.neuralynx_extract.nlxio as nlxio

import preprocessing.data_preprocessing.refractor_utils as ru
import sys
import glob
import os

import matplotlib.pyplot as plt
import datajoint as dj
import scipy as sp
import natsort as ns

# Local application imports 
from database.db_setup import *
import preprocessing.data_preprocessing.binning as binning
import preprocessing.data_preprocessing.create_vectors_from_time_points as create

config


Please enter DataJoint username:  root
Please enter DataJoint password:  ···············


Connecting root@localhost:3306
Epiphyte Setup


In [3]:
top_dir = '/home/anastasia/epiphyte'

columns = ['patient_id', 'session', 'channel_name', 'recording_site', 'brain_area', 'unit_type', 'sampling_freq']

ba_dict = {'LA': 'left amygdala', 'LAH': 'left anterior hippocampus', 'LMH': 'left medial hippocampus', 'LEC': 'left entorhinal cortex',
        'LPHC': 'left parahippocampal cortex', 'RA': 'right amygdala', 'RAH': 'right anterior hippocampus', 'RMH': 'right medial hippocampus',
        'REC': 'right entorhinal cortex', 'RPHC': 'right parahippocampal cortex', 'RAPH':'right anterior parahippocampus',
        'RMPH':'right medial parahippocampus', 'RPPH': 'right posterior parahippocampus'}

In [4]:
patient_id = '53'
ses = 1
#get all .ncs files per subj
ncs_path_list = []
for file in glob.glob(f'{top_dir}/lfp_data/{patient_id}/*.ncs'): #change 46 to sub in a for loop
    ncs_path_list.append(file)

#for each subject here we know the numb of channels so we can preallocate a df for each subj and then merge with the original one

df = pd.DataFrame(columns=columns, index=range(len(ncs_path_list)))
#fill in subj
df['patient_id'] = patient_id
df['session'] = ses

# Specify the file path for a given .ncs file
for i in range(len(ncs_path_list)):
    ncs_path = ncs_path_list[i]
    #instead: get_cscs_for_patient(patient_id, session_nr)
    # Create the ncsobj, which pulls and structures the information from the ncs file 
    ncsobj = ct.ExtractNcsFile(ncs_path)

    # Get header info
    ncs_header = nlxio.ncs_info(ncs_path)

    #fill in df
    temp = ElectrodeUnit()
    df.loc[i,'channel_name'] = ncs_path_list[i][37:-4]
    df.loc[i,'sampling_freq'] = ncs_header['SamplingFrequency']
    df.loc[i,'recording_site'] = ncs_header['AcqEntName']
    df.loc[i,'brain_area'] = ncs_header['AcqEntName']
df['brain_area'] = df['brain_area'].str[:-1]
df['brain_area'] = df['brain_area'].map(ba_dict)

#to solve the issue with names in ch_name
df['channel_name'] = pd.Categorical(df['channel_name'], ordered=True, categories= ns.natsorted(df['channel_name'].unique()))    
df = df.sort_values('channel_name')
df.to_csv(f'{top_dir}/anastasia/output/{patient_id}_channel_info.csv')